In [2]:

import pandas as pd
import numpy as np

import json
import os

import re
import pymongo

In [3]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)  

In [18]:
boston_drinks = pd.read_csv('cocktails_cleaned.csv')
boston_drinks.head()


,name,category,measurement-1,ingredient-1,measurement-2,ingredient-2,measurement-3,ingredient-3,measurement-4,ingredient-4,measurement-5,ingredient-5,measurement-6,ingredient-6,special ingredients,instructions,glass,glass-size (oz)
0,Gauguin,Cocktail Classics,2 oz,Light Rum,1 oz,Passion Fruit Syrup,1 oz,Lemon Juice,1 oz,Lime Juice,NaN,NaN,NaN,NaN,Cherry,Combine ingredients with a cup of crushed ice in blender and blend at low speed. Serve in old-fashioned glass. Garnish with a cherry.,Old-Fashioned Glass,6.0
1,Fort Lauderdale,Cocktail Classics,1.5 oz,Light Rum,.5 oz,Sweet Vermouth,.25 oz,Juice of Orange,.25 oz,Lime Juice,NaN,NaN,NaN,NaN,Orange Slice,Shake with ice and strain into old-fashioned glass over ice cubes. Garnish with a orange slice.,Old-Fashioned Glass,6.0
2,Apple Pie,Cordials and Liqueurs,3 oz,Apple schnapps,1 oz,Cinnamon schnapps,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Apple Slice,Pour into ice-filled old-fashioned glass. Garnish with an apple slice and top with cinnamon.,Old-Fashioned Glass,6.0
3,Cuban Cocktail No. 1,Cocktail Classics,2 oz,Light Rum,.5 oz,Powdered Sugar,.5 oz,Juice of a Lime,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Shake with ice and strain into cocktail glass.,Cocktail Glass,6.0
4,Cool Carlos,Cocktail Classics,1.5 oz,Dark rum,2 oz,Cranberry Juice,2 oz,Pineapple Juice,1 oz,Orange curacao,1 oz,Sour Mix,NaN,NaN,"Pineapple Slice, Orange Slice, Cherry","Mix all ingredients except curacao with ice, shake well. Pour into collins glass and top with curacao. Garnish with a pineapple slice, an orange slice, and a cherry.",Collins Glass,14.0


In [19]:
boston_drinks = boston_drinks.fillna('')

In [20]:
cols = list(boston_drinks)
for i, col in enumerate(cols):
    print(i, col)

0 name
1 category
2 measurement-1
3 ingredient-1
4 measurement-2
5 ingredient-2
6 measurement-3
7 ingredient-3
8 measurement-4
9 ingredient-4
10 measurement-5
11 ingredient-5
12 measurement-6
13 ingredient-6
14 special ingredients
15 instructions
16 glass
17 glass-size (oz)


In [21]:
recipes = []
for index, row in boston_drinks.iterrows():
    name = boston_drinks.loc[index, 'name']
    this_recipe = {}
    this_recipe['name'] = name
    this_recipe['category'] = boston_drinks.loc[index, cols[1]]
    this_recipe['ingredients'] = []
    this_recipe['special_ingredients'] = boston_drinks.loc[index, cols[14]]
    this_recipe['instructions'] = boston_drinks.loc[index, cols[15]]
    if (boston_drinks.loc[index, cols[2]] != '') and (boston_drinks.loc[index, cols[3]] != ''):
        this_recipe['ingredients'].append(str(boston_drinks.loc[index, cols[2]]) + ' ' + str(boston_drinks.loc[index, cols[3]]))
    elif (boston_drinks.loc[index, cols[3]] != ''):
        this_recipe['ingredients'].append(str(boston_drinks.loc[index, cols[3]]))
    if (boston_drinks.loc[index, cols[4]] != '') and (boston_drinks.loc[index, cols[5]] != ''):
        this_recipe['ingredients'].append(str(boston_drinks.loc[index, cols[4]]) + ' ' + str(boston_drinks.loc[index, cols[5]]))
    elif (boston_drinks.loc[index, cols[5]] != ''):
        this_recipe['ingredients'].append(str(boston_drinks.loc[index, cols[3]]))
    if (boston_drinks.loc[index, cols[6]] != '') and (boston_drinks.loc[index, cols[7]] != ''):
        this_recipe['ingredients'].append(str(boston_drinks.loc[index, cols[6]]) + ' ' + str(boston_drinks.loc[index, cols[7]]))
    elif (boston_drinks.loc[index, cols[7]] != ''):
        this_recipe['ingredients'].append(str(boston_drinks.loc[index, cols[7]]))
    if (boston_drinks.loc[index, cols[8]] != '') and (boston_drinks.loc[index, cols[9]] != ''):
        this_recipe['ingredients'].append(str(boston_drinks.loc[index, cols[8]]) + ' ' + str(boston_drinks.loc[index, cols[9]]))
    elif (boston_drinks.loc[index, cols[9]] != ''):
        this_recipe['ingredients'].append(str(boston_drinks.loc[index, cols[9]]))
    if (boston_drinks.loc[index, cols[10]] != '') and (boston_drinks.loc[index, cols[11]] != ''):
        this_recipe['ingredients'].append(str(boston_drinks.loc[index, cols[10]]) + ' ' + str(boston_drinks.loc[index, cols[11]]))
    elif (boston_drinks.loc[index, cols[11]] != ''):
        this_recipe['ingredients'].append(str(boston_drinks.loc[index, cols[11]]))
    if (boston_drinks.loc[index, cols[12]] != '') and (boston_drinks.loc[index, cols[13]] != ''):
        this_recipe['ingredients'].append(str(boston_drinks.loc[index, cols[12]]) + ' ' + str(boston_drinks.loc[index, cols[13]]))
    elif (boston_drinks.loc[index, cols[13]] != ''):
        this_recipe['ingredients'].append(str(boston_drinks.loc[index, cols[13]]))
    this_recipe['glass_type'] = boston_drinks.loc[index, cols[16]]
    this_recipe['glass_size'] = str(boston_drinks.loc[index, cols[17]])
    recipes.append(this_recipe)

In [32]:
recipes

[{'name': 'Gauguin',
  'category': 'Cocktail Classics',
  'ingredients': ['2 oz  Light Rum',
   '1 oz  Passion Fruit Syrup',
   '1 oz  Lemon Juice',
   '1 oz  Lime Juice'],
  'special_ingredients': 'Cherry',
  'instructions': 'Combine ingredients with a cup of crushed ice in blender and blend at low speed. Serve in old-fashioned glass. Garnish with a cherry.',
  'glass_type': 'Old-Fashioned Glass',
  'glass_size': '6.0',
  'total_volume': 5.0,
  '_id': ObjectId('5c954735ad4c601fd8d6669f')},
 {'name': 'Fort Lauderdale',
  'category': 'Cocktail Classics',
  'ingredients': ['1.5 oz  Light Rum',
   '.5 oz  Sweet Vermouth',
   '.25 oz  Juice of Orange',
   '.25 oz  Lime Juice'],
  'special_ingredients': 'Orange Slice',
  'instructions': 'Shake with ice and strain into old-fashioned glass over ice cubes. Garnish with a orange slice.',
  'glass_type': 'Old-Fashioned Glass',
  'glass_size': '6.0',
  'total_volume': 70.0,
  '_id': ObjectId('5c954735ad4c601fd8d666a0')},
 {'name': 'Apple Pie',
  

In [36]:
for recipe in recipes:
    ing_vals = []
    for ing in recipe['ingredients']:
#         print(re.sub(r'\D', '', ing))
        try:
            print('no error')
            print(recipe['name'])
            print(ing_vals)
            ing_vals.append(float(re.sub(r'[A-Za-z]', '', ing)))
            print(ing_vals)
        except:
            print('============')
            print('error')
            print(recipe['name'])
            print(ing_vals)
            print('============')
    recipe['total_volume'] = sum(ing_vals)
    print(sum(ing_vals))

no error
Gauguin
[]
[2.0]
no error
Gauguin
[2.0]
[2.0, 1.0]
no error
Gauguin
[2.0, 1.0]
[2.0, 1.0, 1.0]
no error
Gauguin
[2.0, 1.0, 1.0]
[2.0, 1.0, 1.0, 1.0]
5.0
no error
Fort Lauderdale
[]
[1.5]
no error
Fort Lauderdale
[1.5]
[1.5, 0.5]
no error
Fort Lauderdale
[1.5, 0.5]
[1.5, 0.5, 0.25]
no error
Fort Lauderdale
[1.5, 0.5, 0.25]
[1.5, 0.5, 0.25, 0.25]
2.5
no error
Apple Pie
[]
[3.0]
no error
Apple Pie
[3.0]
[3.0, 1.0]
4.0
no error
Cuban Cocktail No. 1
[]
[2.0]
no error
Cuban Cocktail No. 1
[2.0]
[2.0, 0.5]
no error
Cuban Cocktail No. 1
[2.0, 0.5]
[2.0, 0.5, 0.5]
3.0
no error
Cool Carlos
[]
[1.5]
no error
Cool Carlos
[1.5]
[1.5, 2.0]
no error
Cool Carlos
[1.5, 2.0]
[1.5, 2.0, 2.0]
no error
Cool Carlos
[1.5, 2.0, 2.0]
[1.5, 2.0, 2.0, 1.0]
no error
Cool Carlos
[1.5, 2.0, 2.0, 1.0]
[1.5, 2.0, 2.0, 1.0, 1.0]
7.5
no error
John Collins
[]
[2.0]
no error
John Collins
[2.0]
[2.0, 1.0]
no error
John Collins
[2.0, 1.0]
[2.0, 1.0, 1.0]
no error
John Collins
[2.0, 1.0, 1.0]
[2.0, 1.0, 1.0, 0.5]
4

Hennessy Martini
[]
no error
Hennessy Martini
[]
[0.5]
0.5
no error
Vodka Daisy
[]
[2.0]
no error
Vodka Daisy
[2.0]
[2.0, 1.0]
no error
Vodka Daisy
[2.0, 1.0]
[2.0, 1.0, 0.5]
no error
Vodka Daisy
[2.0, 1.0, 0.5]
[2.0, 1.0, 0.5, 1.0]
4.5
no error
Heavenly Dram
[]
error
Heavenly Dram
[]
no error
Heavenly Dram
[]
[0.5]
no error
Heavenly Dram
[0.5]
[0.5, 0.75]
no error
Heavenly Dram
[0.5, 0.75]
[0.5, 0.75, 0.25]
1.5
no error
Classic Cocktail
[]
[0.25]
no error
Classic Cocktail
[0.25]
[0.25, 1.5]
no error
Classic Cocktail
[0.25, 1.5]
[0.25, 1.5, 1.5]
no error
Classic Cocktail
[0.25, 1.5, 1.5]
[0.25, 1.5, 1.5, 1.0]
4.25
no error
Cherry Sling
[]
error
Cherry Sling
[]
no error
Cherry Sling
[]
[0.5]
0.5
no error
Cherry Fizz
[]
[0.5]
no error
Cherry Fizz
[0.5]
error
Cherry Fizz
[0.5]
no error
Cherry Fizz
[0.5]
error
Cherry Fizz
[0.5]
0.5
no error
Vodka Cooler
[]
[2.0]
no error
Vodka Cooler
[2.0]
[2.0, 0.5]
no error
Vodka Cooler
[2.0, 0.5]
error
Vodka Cooler
[2.0, 0.5]
2.5
no error
Carrol Cocktai

[]
error
Dry Martini
[]
no error
Dry Martini
[]
[0.33]
0.33
no error
Salty Dog
[]
[1.5]
no error
Salty Dog
[1.5]
[1.5, 5.0]
6.5
no error
Dr. Cook
[]
error
Dr. Cook
[]
no error
Dr. Cook
[]
[1.0]
no error
Dr. Cook
[1.0]
[1.0, 1.0]
no error
Dr. Cook
[1.0, 1.0]
[1.0, 1.0, 1.0]
3.0
no error
Dillatini Cocktail
[]
error
Dillatini Cocktail
[]
no error
Dillatini Cocktail
[]
[0.75]
0.75
no error
Chi-Town Flip
[]
[2.0]
no error
Chi-Town Flip
[2.0]
[2.0, 0.75]
no error
Chi-Town Flip
[2.0, 0.75]
[2.0, 0.75, 0.75]
no error
Chi-Town Flip
[2.0, 0.75, 0.75]
error
Chi-Town Flip
[2.0, 0.75, 0.75]
no error
Chi-Town Flip
[2.0, 0.75, 0.75]
[2.0, 0.75, 0.75, 0.25]
no error
Chi-Town Flip
[2.0, 0.75, 0.75, 0.25]
[2.0, 0.75, 0.75, 0.25, 1.0]
4.75
no error
Chef's Pain
[]
[2.0]
no error
Chef's Pain
[2.0]
[2.0, 0.75]
no error
Chef's Pain
[2.0, 0.75]
[2.0, 0.75, 0.5]
no error
Chef's Pain
[2.0, 0.75, 0.5]
error
Chef's Pain
[2.0, 0.75, 0.5]
3.25
no error
Sampan Shipwreck A
[]
[1.5]
no error
Sampan Shipwreck A
[1.5]
[

[]
[0.5]
no error
Chocolate Soldier
[0.5]
[0.5, 0.75]
no error
Chocolate Soldier
[0.5, 0.75]
[0.5, 0.75, 1.5]
2.75
no error
Chocolate Flip
[]
[1.0]
no error
Chocolate Flip
[1.0]
[1.0, 1.0]
no error
Chocolate Flip
[1.0, 1.0]
[1.0, 1.0, 0.75]
no error
Chocolate Flip
[1.0, 1.0, 0.75]
[1.0, 1.0, 0.75, 0.75]
no error
Chocolate Flip
[1.0, 1.0, 0.75, 0.75]
error
Chocolate Flip
[1.0, 1.0, 0.75, 0.75]
3.5
no error
Chocolate Cocktail
[]
[1.5]
no error
Chocolate Cocktail
[1.5]
[1.5, 1.5]
no error
Chocolate Cocktail
[1.5, 1.5]
[1.5, 1.5, 1.0]
no error
Chocolate Cocktail
[1.5, 1.5, 1.0]
[1.5, 1.5, 1.0, 1.0]
5.0
no error
Chicago Fizz
[]
[0.5]
no error
Chicago Fizz
[0.5]
[0.5, 1.0]
no error
Chicago Fizz
[0.5, 1.0]
[0.5, 1.0, 1.0]
no error
Chicago Fizz
[0.5, 1.0, 1.0]
[0.5, 1.0, 1.0, 1.0]
no error
Chicago Fizz
[0.5, 1.0, 1.0, 1.0]
[0.5, 1.0, 1.0, 1.0, 1.0]
4.5
no error
Pretty In Pink
[]
[2.0]
no error
Pretty In Pink
[2.0]
[2.0, 0.5]
no error
Pretty In Pink
[2.0, 0.5]
[2.0, 0.5, 0.75]
no error
Pretty I

Lights On The Plaza
[0.5, 0.5, 1.5, 0.5]
3.0
no error
Rococo
[]
[1.0]
no error
Rococo
[1.0]
[1.0, 1.0]
no error
Rococo
[1.0, 1.0]
[1.0, 1.0, 0.5]
2.5
no error
Rock & Rye Cooler
[]
[1.0]
no error
Rock & Rye Cooler
[1.0]
[1.0, 1.5]
no error
Rock & Rye Cooler
[1.0, 1.5]
[1.0, 1.5, 1.0]
no error
Rock & Rye Cooler
[1.0, 1.5, 1.0]
error
Rock & Rye Cooler
[1.0, 1.5, 1.0]
3.5
no error
Rock & Rye Cocktail
[]
[1.0]
no error
Rock & Rye Cocktail
[1.0]
[1.0, 1.0]
no error
Rock & Rye Cocktail
[1.0, 1.0]
[1.0, 1.0, 1.5]
3.5
no error
Le Paradini
[]
[1.5]
no error
Le Paradini
[1.5]
error
Le Paradini
[1.5]
no error
Le Paradini
[1.5]
[1.5, 0.5]
no error
Le Paradini
[1.5, 0.5]
[1.5, 0.5, 1.0]
3.0
no error
Red Swizzle
[]
[1.0]
no error
Red Swizzle
[1.0]
[1.0, 1.0]
no error
Red Swizzle
[1.0, 1.0]
[1.0, 1.0, 2.0]
no error
Red Swizzle
[1.0, 1.0, 2.0]
[1.0, 1.0, 2.0, 2.0]
no error
Red Swizzle
[1.0, 1.0, 2.0, 2.0]
[1.0, 1.0, 2.0, 2.0, 2.0]
no error
Red Swizzle
[1.0, 1.0, 2.0, 2.0, 2.0]
[1.0, 1.0, 2.0, 2.0, 2.0,

no error
Frozen Apple
[]
[1.5]
no error
Frozen Apple
[1.5]
[1.5, 1.0]
no error
Frozen Apple
[1.5, 1.0]
error
Frozen Apple
[1.5, 1.0]
2.5
no error
Frisky Witch
[]
[1.0]
no error
Frisky Witch
[1.0]
[1.0, 1.0]
2.0
no error
Froth Blower Cocktail
[]
[1.0]
no error
Froth Blower Cocktail
[1.0]
[1.0, 2.0]
3.0
no error
French Martini
[]
[1.5]
no error
French Martini
[1.5]
[1.5, 0.75]
2.25
no error
Flatiron Martini
[]
[1.0]
no error
Flatiron Martini
[1.0]
error
Flatiron Martini
[1.0]
1.0
no error
Favourite Cocktail
[]
error
Favourite Cocktail
[]
no error
Favourite Cocktail
[]
[0.75]
no error
Favourite Cocktail
[0.75]
error
Favourite Cocktail
[0.75]
0.75
no error
Electric Jam
[]
[1.25]
no error
Electric Jam
[1.25]
[1.25, 0.5]
no error
Electric Jam
[1.25, 0.5]
error
Electric Jam
[1.25, 0.5]
no error
Electric Jam
[1.25, 0.5]
error
Electric Jam
[1.25, 0.5]
1.75
no error
Fairy Belle Cocktail
[]
[1.0]
no error
Fairy Belle Cocktail
[1.0]
[1.0, 1.0]
no error
Fairy Belle Cocktail
[1.0, 1.0]
error
Fairy B

Robson Cocktail
[1.5, 0.5]
error
Robson Cocktail
[1.5, 0.5]
2.0
no error
Rio Fix
[]
[2.0]
no error
Rio Fix
[2.0]
[2.0, 1.5]
no error
Rio Fix
[2.0, 1.5]
error
Rio Fix
[2.0, 1.5]
no error
Rio Fix
[2.0, 1.5]
[2.0, 1.5, 0.5]
4.0
no error
Ringo Starr
[]
[3.0]
no error
Ringo Starr
[3.0]
[3.0, 0.5]
no error
Ringo Starr
[3.0, 0.5]
error
Ringo Starr
[3.0, 0.5]
no error
Ringo Starr
[3.0, 0.5]
[3.0, 0.5, 2.0]
5.5
no error
Restless Native
[]
[2.0]
no error
Restless Native
[2.0]
[2.0, 0.75]
no error
Restless Native
[2.0, 0.75]
[2.0, 0.75, 1.5]
4.25
no error
Quarter Deck Cocktail
[]
[1.5]
no error
Quarter Deck Cocktail
[1.5]
[1.5, 0.5]
2.0
no error
Quaker's Cocktail
[]
[0.75]
no error
Quaker's Cocktail
[0.75]
[0.75, 0.75]
no error
Quaker's Cocktail
[0.75, 0.75]
error
Quaker's Cocktail
[0.75, 0.75]
1.5
no error
Puerto Rican Rum Daisy
[]
[1.5]
no error
Puerto Rican Rum Daisy
[1.5]
[1.5, 0.25]
no error
Puerto Rican Rum Daisy
[1.5, 0.25]
error
Puerto Rican Rum Daisy
[1.5, 0.25]
no error
Puerto Rican Rum

[]
[1.0]
no error
Sunday Confession
[1.0]
[1.0, 1.0]
no error
Sunday Confession
[1.0, 1.0]
[1.0, 1.0, 0.5]
2.5
no error
Strawberry Margarita
[]
error
Strawberry Margarita
[]
no error
Strawberry Margarita
[]
error
Strawberry Margarita
[]
no error
Strawberry Margarita
[]
[0.75]
no error
Strawberry Margarita
[0.75]
[0.75, 0.75]
1.5
no error
Western Rose Cocktail
[]
error
Western Rose Cocktail
[]
no error
Western Rose Cocktail
[]
error
Western Rose Cocktail
[]
no error
Western Rose Cocktail
[]
error
Western Rose Cocktail
[]
0
no error
Widow's Kiss Cocktail
[]
error
Widow's Kiss Cocktail
[]
no error
Widow's Kiss Cocktail
[]
[0.5]
no error
Widow's Kiss Cocktail
[0.5]
error
Widow's Kiss Cocktail
[0.5]
0.5
no error
Spice Of Life
[]
[3.0]
no error
Spice Of Life
[3.0]
[3.0, 12.0]
no error
Spice Of Life
[3.0, 12.0]
error
Spice Of Life
[3.0, 12.0]
no error
Spice Of Life
[3.0, 12.0]
[3.0, 12.0, 0.75]
15.75
no error
Widow's Dream Cocktail
[]
[1.5]
no error
Widow's Dream Cocktail
[1.5]
[1.5, 1.0]
2.5

[1.5, 0.5, 0.5, 1.0]
[1.5, 0.5, 0.5, 1.0, 1.5]
5.0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
no error
Bucks Fizz
[]
[2.0]
no error
Bucks Fizz
[2.0]
error
Bucks Fizz
[2.0]
2.0
0
0
no error
Count Camillo's Paloma
[]
[0.75]
no error
Count Camillo's Paloma
[0.75]
[0.75, 0.75]
no error
Count Camillo's Paloma
[0.75, 0.75]
[0.75, 0.75, 0.75]
no error
Count Camillo's Paloma
[0.75, 0.75, 0.75]
[0.75, 0.75, 0.75, 2.0]
4.25
no error
Company B
[]
error
Company B
[]
no error
Company B
[]
[1.5]
no error
Company B
[1.5]
[1.5, 0.5]
no error
Company B
[1.5, 0.5]
[1.5, 0.5, 0.5]
no error
Company B
[1.5, 0.5, 0.5]
[1.5, 0.5, 0.5, 0.5]
no error
Company B
[1.5, 0.5, 0.5, 0.5]
[1.5, 0.5, 0.5, 0.5, 0.25]
3.25
no error
Chupa Cabra
[]
[2.0]
no error
Chupa Cabra
[2.0]
[2.0, 0.75]
no error
Chupa Cabra
[2.0, 0.75]
[2.0, 0.75, 0.5]
no error
Chupa Cabra
[2.0, 0.75, 0.5]
[2.0, 0.75, 0.5, 0.5]
no error
Chupa Cabra
[2.0, 0.75, 0.5, 0.5]
[2.0, 0.75, 0.5, 0.5, 0.5]
no error
Chupa Cabra
[2.0

In [37]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
mongoDb = client.cocktail_db

collection = mongoDb.recipe_dump

collection.insert_many(recipes)

In [29]:
cocktail_db_response = collection.find({}, {'_id': False})


In [30]:
recipes = []
for recipe in cocktail_db_response:
    recipes.append(recipe)


In [31]:
recipes

[{'name': 'Gauguin',
  'ingredients': ['2 oz  Light Rum',
   '1 oz  Passion Fruit Syrup',
   '1 oz  Lemon Juice',
   '1 oz  Lime Juice'],
  'instructions': 'Combine ingredients with a cup of crushed ice in blender and blend at low speed. Serve in old-fashioned glass. Garnish with a cherry.',
  'glass_type': 'Old-Fashioned Glass',
  'glass_size': '6'},
 {'name': 'Fort Lauderdale',
  'ingredients': ['1 1/2 oz  Light Rum',
   '1/2 oz  Sweet Vermouth',
   '1/4 oz  Juice of Orange',
   '1/4 oz  Juice of a Lime'],
  'instructions': 'Shake with ice and strain into old-fashioned glass over ice cubes. Garnish with a orange slice.',
  'glass_type': 'Old-Fashioned Glass',
  'glass_size': '6'},
 {'name': 'Apple Pie',
  'ingredients': ['3 oz  Apple schnapps',
   '1 oz  Cinnamon schnapps',
   ' Apple slice'],
  'instructions': 'Pour into ice-filled old-fashioned glass. Garnish with an apple slice and top with cinnamon.',
  'glass_type': 'Old-Fashioned Glass',
  'glass_size': '6'},
 {'name': 'Cuban C